In [ ]:
%reload_ext autoreload
%autoreload 2

from util.scenario import Scenario
import numpy as np
import matplotlib.pyplot as plt
from pyomo_models.base import BaseModel
from pyomo_models.multi_uavs import MultiUavModel
from pyomo.opt import SolverFactory
from simulate.simulate import Simulation
from util import constants
from tqdm.notebook import tqdm

# Base model

In [ ]:
sc = Scenario.from_file("scenarios/single_longer_path.yml")

_, ax = plt.subplots(figsize=(6,6), dpi=80)
sc.plot(ax=ax, draw_distances=False)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
# PARAMETERS
B_min = 0.1
B_max = 1
B_start = [1]
v = [1]
r_charge = np.array([0.2])
r_deplete = np.array([0.3])

In [ ]:
parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
parameters['epsilon'] = 0.1
model = BaseModel(sc, parameters)

In [ ]:
solver = SolverFactory("mindtpy")
solution = solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt')

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.C[:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")

In [ ]:
axes = plt.subplots(1,2, figsize=(10,20), dpi=80) 
ax1, ax2= axes[1]

# problem
sc.plot(ax=ax1, draw_distances=False)
ax1.set_aspect('equal', adjustable='box')
ax1.axis('off')

model.plot(ax=ax2)
ax2.set_aspect('equal', adjustable='box')
ax2.axis('off')

plt.savefig("out/example_path.pdf", bbox_inches="tight")

plt.show()

In [ ]:
_, ax = plt.subplots(figsize=(6,1), dpi=100)
model.plot_charge(0, ax=ax, color='red')

ax.set_xticks([])
ax.set_xlabel("Time")

plt.savefig("out/example_battery_profile.pdf", bbox_inches="tight")
plt.show()

# Multi-UAV ($\epsilon$ impact)

In [ ]:
sc = Scenario.from_file("scenarios/three_drones_circling.yml")
# sc = Scenario.from_file("scenarios/two_longer_path.yaml")

_, ax = plt.subplots(figsize=(4,4), dpi=70)
sc.plot(ax=ax, draw_distances=False)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')

plt.savefig("out/multi_uav_path.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# PARAMETERS
B_min = 0.1
B_max = 1
B_start = [1, 1, 1]
v = [1,1.2,1]
r_charge = np.array([0.15, 0.15, 0.15])
r_deplete = np.array([0.3, 0.3, 0.3])

# CREATE MODEL
models = []
epsilons = [0.1, 1]
for epsilon in epsilons:
    parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'v', 'r_charge', 'r_deplete']}
    parameters['epsilon'] = epsilon
    model = MultiUavModel(sc, parameters)
    models.append(model)

In [ ]:
solver = SolverFactory("mindtpy")
for model in tqdm(models):
    solution = solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt')

In [ ]:
max_time = 0
for model in models:
    for d in model.d:
        sim = Simulation.from_base_model(model, d).simulate()        
        t = sim[1][-1]
        max_time = max(max_time, t)

for i, model in enumerate(models):    
    fig, axes = plt.subplots(model.N_d, 1, figsize=(3,model.N_d), dpi=80, sharey=True)
    fig.subplots_adjust(hspace=.4)
    
    for d in model.d:        
        sim = Simulation.from_base_model(model, d)
        sim.plot_charge(ax=axes[d], color=constants.W_COLORS[d])
        # model.plot_charge(d, ax=axes[d], color=constants.W_COLORS[d])
        
        # set xtick
        t = sim.simulate()[1][-1]
        axes[d].set_xticks([t], [f"{t:.2f}s"])

        axes[d].set_xlim([-.5, max_time+0.5])
        if d != model.N_d - 1:
            axes[d].set_xlabel("")
    
    axes[model.N_d-1].set_xlabel("Time")

    plt.savefig(f"out/multi_uav_battery_{i}.pdf", bbox_inches="tight")
    plt.savefig(f"out/multi_uav_battery_{i}.png", dpi=500, bbox_inches="tight")
plt.show()